In [1]:
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import scipy.special as special
from scipy.optimize import curve_fit
from tabulate import tabulate

In [2]:
%matplotlib inline

In [ ]:
ll_vac_adr = tkf.askopenfilename()
print(ll_vac_adr)
ll_vac = hys.load(ll_vac_adr, signal_type="EELS")
print(ll_vac)

In [ ]:
ll_vac.align_zero_loss_peak()

In [3]:
ll_adr = tkf.askopenfilename()
print(ll_adr)
ll = hys.load(ll_adr, signal_type="EELS")
print(ll)

C:/Users/RYUEMSL/Desktop/RYU/temporary/report/low_loss/20191016_low_loss_Si_GaN/Si_0.01sec_100frames_mono1.0_aperture2.5mm_dispersion0.01.dm3
[<EELSSpectrum, title: Inelastic Spectrum, dimensions: (|2048)>, <EELSSpectrum, title: Inelastic Spectrum, dimensions: (|2048)>, <EELSSpectrum, title: Inelastic Spectrum, dimensions: (|2048)>]


In [4]:
ll = ll[0]

In [5]:
ll.align_zero_loss_peak()


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	0
std:	0

min:	0
Q1:	0
median:	0
Q3:	0
max:	0


In [6]:
def beta_function(a, b):
    return special.gamma(a) * special.gamma(b) / special.gamma(a*b)

def pearson7(x, nor, alpha, lmb, m, cor_term):
    return nor*np.power((1+np.power(((x-lmb)/alpha), 2)), -m) / (alpha * beta_function(m-0.5, 0.5) + cor_term)

In [7]:
def cauchy(x, nor, gamma, x0, cor_term):
    return nor*gamma**2 / (np.pi*gamma*(np.power((x-x0), 2)+gamma**2)) + np.exp(cor_term)

In [12]:
%matplotlib qt
step = ll.axes_manager[0].scale
left = ll.axes_manager[0].offset
e_size = ll.axes_manager[0].size
print(left, step, e_size)
x_range = np.arange(left, (e_size)*step+left, step)
print(x_range[0], x_range[-1])
print(x_range.shape)

plt.figure(figsize=(10,10))

ll_zlp_removed = ll.isig[left:]
print(ll_zlp_removed.axes_manager[0].offset, ll_zlp_removed.axes_manager[0].scale)
temp = ll_zlp_removed.data.copy()
print(temp.shape)
plt.plot(x_range, temp, "k-", label="original spectrum")

maxid = np.argmax(temp)
print(maxid)
left_side = temp[:(maxid+1)].copy()
zlp_mirror = np.zeros(maxid*2+1)
zlp_mirror[:(maxid+1)] = left_side
zlp_mirror[maxid:] = np.flip(left_side)
print(zlp_mirror.shape)

e_range = np.arange(left, len(zlp_mirror)*step+left, step)
#plt.plot(e_range, temp[:len(zlp_mirror)], label="cropped zlp")
popt_o, pcov_o = curve_fit(pearson7, e_range, temp[:len(zlp_mirror)])
fitted_o = pearson7(x_range, *popt_o)
#popt_o, pcov_o = curve_fit(cauchy, e_range, temp[:len(zlp_mirror)])
#fitted_o = cauchy(x_range, *popt_o)
plt.plot(x_range, fitted_o, label="ZLP fitting (modified Pearson 7)", alpha=1.0)
plt.plot(x_range, ll_zlp_removed.data - fitted_o, label="ZLP subtracted spectrum", alpha=1.0)

#plt.plot(e_range, zlp_mirror, label="mirrored ZLP", alpha=0.5)
#ll_zlp_removed_mirror = temp.copy()
#ll_zlp_removed_mirror[:len(zlp_mirror)] -= zlp_mirror
#plt.plot(x_range, ll_zlp_removed_mirror, label="ZLP subtracted spectrum (ZLP mirror)", alpha=0.5)
plt.legend(fontsize="xx-large")
plt.grid()
plt.show()

-1.8599999584257603 0.009999999776482582 2048
-1.8599999584257603 18.609999584034085
(2048,)
-1.8599999584257603 0.009999999776482582
(2048,)
186
(373,)


In [ ]:
ll_vac_rs = ll_vac.deepcopy()
ll_vac_rs.data = ll_vac_rs * np.max(ll.data) / np.max(ll_vac.data)

In [ ]:
%matplotlib qt
hys.plot.plot_spectra([ll, ll_vac_rs], legend="auto")

In [ ]:
ll_zlp_removed = ll.isig[0.0:10.0]-ll_vac_rs.isig[0.0:10.0]
print(ll_zlp_removed)

In [ ]:
%matplotlib qt
hys.plot.plot_spectra([ll.isig[-1.0:10.0], ll_vac_rs.isig[-1.0:10.0], ll_zlp_removed], legend="auto")

In [ ]:
eps = ll_zlp_removed.kramers_kronig_analysis(zlp=ll_zlp_removed, n=3.9766)
print(eps)

In [ ]:
%matplotlib qt
hys.plot.plot_spectra([eps.real, eps.imag], legend="auto")

In [ ]:
ll.save(ll_adr[:-4]+"_zlp_aligned.hdf5")

In [ ]:
ll_cropped = ll.isig[0.0:10.0]
ll_cropped.save(ll_adr[:-4]+"_zlp_aligned_cropped.hdf5")

In [ ]:
ll_log = ll.fourier_log_deconvolution(ll, add_zlp=True)

In [ ]:
%matplotlib qt
ll_log.plot()

In [ ]:
ll_ratio = ll.fourier_ratio_deconvolution(ll_vac)

In [ ]:
ll_ratio.plot()